<a href="https://colab.research.google.com/github/pld000/z3/blob/main/z3_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# IV. Продвинутые темы

*Подготовлено Высшей школой программирования Сергея Бобровского 2023*

https://vk.com/lambda_brain

Ссылки на оригинальные ноутбуки в первом занятии.

In [ ]:
!pip install "z3-solver"
from z3 import *

# 1. Выражения, сорта и декларации


Выражения, сорта и декларации в Z3 доступны как ASTs (имеется в виду абстрактное синтаксическое дерево, АСТ). AST -- это по сути направленнный ациклический граф. Каждое выражение имеет свой сорт (тип). Метод sort() выдаёт сорт выражения.

In [ ]:
x = Int('x')
y = Real('y')
print ((x + 1).sort()) # сорт Int
print ((y + 1).sort()) # сорт Real
print ((x >= 2).sort()) # сорт Bool

Функция `eq(n1, n2)` возвращает True, если n1 и n2 имеют одно и то же AST. Это по сути тест на полную эквивалентность внутренних структур выражений.

In [ ]:
x, y = Ints('x y')
print (eq(x + y, x + y))
print (eq(x + y, y + x)) # False: по обе стороны от знака операции расположены разные идентификаторы

n = x + y
print (eq(n, x + y))

x2 = Int('x')
print (eq(x, x2)) # сравнение x с самим собой

print (eq(Int('x'), Real('x'))) # дробная часть не равна целой части

Сравнение узлов выполняется на основе метода `hash()` (хэш-код для узла AST). Если хэши n1.hash() b n2.hash() равны, то eq(n1, n2) выдаст True.

In [ ]:
x = Int('x')
print ((x + 1).hash())
print ((1 + x).hash())
print (x.sort().hash())

Выражения Z3 делятся на три базовые группы: **приложения, кванторы и связанные/свободные переменные**.

**Приложений** достаточно для полноценного решения, если задача не содержит **квантора** всеобщности/существования. **Квантор всеобщности** для x требует, чтобы условие выполнялось для всех значений x, а **квантор существования** для x требует, чтобы условие выполнялось хотя бы для одного значения x.

В Z3 формально не существует переменных, технически это **константы**, которые внутри движка представляются как функции (приложения) с 0 аргументов. Например, Int('x') -- это фактически объявление целочисленной функции x().

Каждое приложение ассоциируется со своей **декларацией** (ссылкой, в некотором смысле), и содержит 0 или более аргументов. Метод decl() возвращает связанную с приложением декларацию. Метод num_args() возвращает количество аргументов приложения, а `arg(i)` -- i-й аргумент. Функция `is_expr(n)` возвращает True, если n -- выражение. Соответственно, `is_app(n)` (is_func_decl(n)) возвращает True, если n -- приложение (декларация).

In [ ]:
x = Int('x')
print ("is expression: ", is_expr(x))
n = x + 1
print ("is application:", is_app(n))
print ("decl:          ", n.decl())
print ("num args:      ", n.num_args())
for i in range(n.num_args()):
    print ("arg(", i, ") ->", n.arg(i))

Декларации имеют имена, доступные через метод `name()`. Декларация имеет арность (количество параметров), домен (типы параметров) и диапазон сортов.

In [ ]:
x   = Int('x') # x - переменая
x_d = x.decl() # декларация переменной
print ("is_expr(x_d):     ", is_expr(x_d)) # переменная -- не выражение
print ("is_func_decl(x_d):", is_func_decl(x_d))
print ("x_d.name():       ", x_d.name()) # просто x
print ("x_d.range():      ", x_d.range()) # один тип Int
print ("x_d.arity():      ", x_d.arity()) # 0 аргументов

# вызов декларации как функции -- x_d() вернёт оригинальное приложение
print ("eq(x_d(), x):     ", eq(x_d(), x))


In [ ]:
f   = Function('f', IntSort(), RealSort(), BoolSort()) # f - функция с параметрами (Int, Real) и результатом Bool
print ("f.name():         ", f.name())
print ("f.range():        ", f.range()) # Bool
print ("f.arity():        ", f.arity()) # 2
for i in range(f.arity()):
    print ("domain(", i, "): ", f.domain(i)) # Int, Real

print (f(x, x)) # приложение f(x, ToReal(x)) с двумя аргументами
print (eq(f(x, x).decl(), f)) # True

Встроенные декларации идентифицируются по их **роду** (виду, классу, типу, разновидности -- kind). Он доступен через метод kind(). Полный список всех встроенных деклараций доступен в файле z3consts.py (z3_api.h) в дистрибутиве Z3.

In [ ]:
x, y = Ints('x y')
print ((x + y).decl().kind() == Z3_OP_ADD) # это сложение
print ((x + y).decl().kind() == Z3_OP_SUB)

Выражения можно упрощать через замену подвыражений с помощью функции `substitute`.

In [ ]:
x, y = Ints('x y')
f    = Function('f', IntSort(), IntSort(), IntSort())
g    = Function('g', IntSort(), IntSort())

# выражение n -- комбинация двух функций f и g
n    = f(f(g(x), g(g(x))), g(g(y)))
print (n)

# заменим g(g(x)) на y, и g(y) -- на x + 1
print (substitute(n, (g(g(x)), y), (g(y), x + 1)))

# 2. Массивы


## 2.1. Декларация массивов

Массив вводится в программе с помощью ключевого слова `Array`. Первый параметр конструктора -- имя массива, второй -- тип индексов, третий -- тип значений.

In [ ]:
I = IntSort()
A = Array('A', I, I) # A -- массив, хранящий целые числа
x = Int('x')
print (A[x])
print (Select(A, x))
print (Store(A, x, 10))
print (simplify(Select(Store(A, 2, x+1), 2))) # извлечь значение 2-го элемена, куда предварительно записать x+1

Более компактно массив некоторого типа можно задавать с помощью `Vector`, указывая тип как префикс.

In [ ]:

X = Array('x', IntSort(), IntSort())
print (X[0] + X[1] + X[2] >=0) # ограничение на сумму трёх элементов массива (не лучший способ)

X = IntVector('x', 3) # вектор из 3 целочисленных элементов
print (X[0] + X[1] + X[2] >= 0)
print (Sum(X) >= 0) # лучший способ задать соответствующее ограничение

## 2.2. Select и Store

В математической теории вычислений Джона Маккарти (автор термина "Искусственный интеллект" и языка Лисп) сформулирована базовая теория массивов, которая обычно задаётся двумя аксиомами. В них используются две операции:

-- Select(a, i) возвращает значение, хранящееся в позиции i массива a.
В нотации Z3 данную операцию можно записать как a[i];

-- Store(a, i, v) возвращает новый массив, идентичный массиву a, но только в позиции i он содержит значение v.

Первая аксиома подразумевает, что если i = j, то при Select(Store(A,x,v), j) мы получим v (записали в массив A значение v в позицию i, и затем считываем значение из результирующего массива A в позиции j, равной i).

Пусть A -- массив целых, тогда условия A[x] == x и Store(A, x, y) == A истинны для массива, где x "разложены" по x, и когда x == y. Это можно доказать так:

In [ ]:
A = Array('A', IntSort(), IntSort())
x, y, v = Ints('x y v')
prove(Implies(x == y,  Select(Store(A,x,v), y) == v))

И вторая аксиома на случай, когда x != y:

Select(Store(A,x,v), y) должна равняться, очевидно, y-му значению A.

In [ ]:
prove(Implies(x != y,  Select(Store(A,x,v), y) == A[y]))


In [ ]:
# Пример
Z = IntSort()
f = Function('f', Z, Z)
x, y, z = Ints('x y z')
A = Array('A',Z,Z)
prove(Implies(x + 2 == y, f(Store(A, x, 3)[y - 2]) == f(y - x + 1)))

# 3. Кванторы

## 3.1. Количественные характеристики высказываний

Мы видели, что Z3 может решать задачи из логики высказываний (в частности, связанные с арифметикой и логическими значениями, битовыми векторами, массивами, функциями и даже алгебраическими типами данных), где не используются кванторы.

Однако Z3 также позволяет квантифицировать высказывания -- добавлять им количественные характеристики (в результате получается логика предикатов). Однако в общем случае не существует универсальной процедуры разрешения таких высказываний (как и в случае логики первого порядка).


Квантор всеобщности требует, чтобы высказывание было истинным для всех без исключения значений заданных переменных.

In [ ]:
f = Function('f', IntSort(), IntSort(), IntSort()) # функция f получает два целочисленных аргумента и возвращает целый результат
x, y = Ints('x y')

# Найти функцию f(x,y) такую, что для всех значений x и y она будет возвращать 0
solve(ForAll([x,y], f(x, y) == 0))


Получаем функцию `f = [else -> 0]` , что означает, что независимо от значений аргументов она всегда возвращает 0. Очевидно, она удовлетворяет условиям.

Квантор существования требует, чтобы высказывание было истинным хотя бы для одного значения заданных переменных.

In [ ]:
# Найти функцию f(x,y) такую, что f(x,x) >= 0 при хотя бы одном значении x
solve (Exists(x, f(x, x) >= 0))

И тут будет построена такая же функция, которая также будет истинной для заданных ограничений.

In [ ]:
# Одно из свойств квантора всеобщности:
prove(Implies(ForAll(y, ForAll(x, (f(x, y) == x + y))), ForAll(x, ForAll(y, (f(x, y) == x + y)))))
prove(Implies(ForAll([x, y], f(x, y) == x + y), ForAll(x, ForAll(y, (f(x, y) == x + y)))))

## 3.2. Моделирование ООП с кванторами

Построим простую модель объектно-ориентированной системы типов с единственным наследованием. В качестве её базы используется концепция subtyping (подтипизация), которую часто путают с наследованием. Subtyping -- это более универсальный принцип подстановки типов (везде, где может использоваться значение родительского типа, может задаваться и значение дочерних типов). Наследование -- более "тяжёлая" прикладная схема, ориентированная на наследование классов вместе с их реализацией, с учётом видимости,  и т. п.

Будет, в частности, использована ковариантность в виде сущности array_of: массив элементов типа-потомка может использоваться везде, где может использоваться массив элементов родительского типа.


In [ ]:
Type     = DeclareSort('Type') # "объектный" тип
subtype  = Function('subtype', Type, Type, BoolSort()) # подтипизация: второй параметр (тип) есть "наследник" первого параметра (типа)
array_of = Function('array_of', Type, Type) # прикладная ковариативная сущность
root     = Const('root', Type) # root -- базовый, корневой тип во всей системе типов

x, y, z  = Consts('x y z', Type)

axioms = [ ForAll(x, subtype(x, x)), # любой тип есть подтип самого себя

           # если y - подтип x, и x - подтип y, то x и y -- один и тот же тип
           ForAll([x, y], Implies(And(subtype(x, y), subtype(y, x)),
                                  x == y)),

           # если y - подтип x, и z - подтип y, то z - подтип x
           ForAll([x, y, z], Implies(And(subtype(x, y), subtype(y, z)),
                                     subtype(x, z))),

           # если y и z -- подтипы x, то y и z будут подтипами друг друга в том смысле,
           # что это не наследование, а именно подстановка типов:
           # где может указываться y, там же может указываться и z
           ForAll([x, y, z], Implies(And(subtype(x, y), subtype(x, z)),
                                     Or(subtype(y, z), subtype(z, y)))),

           # ковариантность array_of
           ForAll([x, y], Implies(subtype(x, y),
                                  subtype(array_of(x), array_of(y)))),

           ForAll(x, subtype(root, x)) # root -- корневой тип, вместо которого могут подставляться все другие типы
           ]

s = Solver()
s.add(axioms)
print (s.check()) # sat!
print (s.model()) # реализация

## 3.3. Индуктивное доказательство бесконечных сумм

Когда требуются доказательства в отношении бесконечных структур, стандартных возможностей Z3 не всегда будет достаточно. Потребуется вручную задавать некоторую форму индуктивных рассуждений.

Например, имеется формула суммы $\sum_{i=0}^n i = \frac{n(n+1)}{2}$

Попробуем её доказать в лоб -- поиском контрпримера.


In [ ]:
n = Int("n")
Sumn = Function("sumn", IntSort(), IntSort())
s = Solver()
s.add( Sumn(0) == 0)
s.add(ForAll([n], Sumn(n+1) == n + 1 + Sumn(n)))
s.add(Not(ForAll([n], Implies(n >= 0, 2 * Sumn(n) == n * (n + 1)))))
s.check()

Z3 с этим самостоятельно не справится, потому что задача достаточно сложная, и на практике тут применяются более мощные инструменты, наподобие интерактивных ассистентов доказательств наподобие Coq. Тут солверу нужны подсказки, например в индуктивном формате.

In [ ]:
n = Int("n")
Sumn = Function("sumn", IntSort(), IntSort())
s = Solver()
s.add( Sumn(0) == 0)
s.add(ForAll([n], Sumn(n+1) == n + 1 + Sumn(n)))

# определим саму Summ индуктивно, без явной рекурсии
def indSumn(n):
    return 2 * Sumn(n) == n * (n + 1)

# доказательство по индукции
def induction(p):
    n = Int("n")
    return Implies(And(  p(0), # если истинна indSumn(0) ...
                         ForAll([n],Implies(And(n >= 0, p(n)),p(n+1)))), # и для всех n и истинности indSumn(n) следует истинность indSumn(n+1)

                       ForAll( [n] , Implies(n>=0, p(n)))) # то получаем истинность indSumn(n)

s.add(induction(indSumn))
s.add( Not(ForAll([n], Implies( n >= 0, indSumn(n))))) # пытаемся доказать противоположное -- найти контрпример
print(s.check()) # для успеха нужен unsat

## 3.4. Логика указателей

In [ ]:
Ptr = DeclareSort('Ptr') # тип Ptr - указатель/ссылка на значение
ref = Function('ref', Z, Ptr) # ref - получить ссылку на значение
deref = Function('deref', Ptr, Z) # deref - по ссылке вытащить значение

Реализуем на указателях сортированную "кучу" -- набор значений, которые хранятся в упорядоченном виде, что обеспечивает быстрый поиск по значению.

In [ ]:
n, x, x1, x2 = Ints('n x x1 x2')

s = Solver()

s.add(ForAll([x1,x2],
             Implies(And(0 <= x1, x1 <= x2, x2 <= n), # для всех неотрицательных значений x1 и x2, если x2 >= x1
                 deref(ref(x1)) <= deref(ref(x2))))) # значение, полученное по ссылке на x1, будет <= значения, полученного по ссылке на x2

p0 = Const('p0', Ptr) # "нулевой" указатель null/None

# в диапазоне неотрицательных значений от 0 до n нулевой указатель отсутствует
s.add(ForAll(x, Implies(And(0 <= x, x <= n), ref(x) != p0)))

print(s.check())

# 4. Оптимизация

Z3 содержит встроенный оптимизатор. Под оптимизацией понимается не какая-то внутренняя оптимизация процесса решения, а классические оптимизационные задачки. Например, не просто найти решение, а максимальное или минимальное значения -- это типичная оптимизационная задача.

Классический пример из линейного программирования.

In [ ]:
a = Real('a')
b = Real('b')
c = Real('c')
d = Real('d')
e = Real('e')
g = Real('g')
f = Real('f')
cost = Real('cost')

opt = Optimize() # создаём солвер-оптимизатор

# набор ограничений
opt.add(a + b - 350 == 0)
opt.add(a - g == 0)
opt.add(c - 400 == 0)
opt.add(b - d * 0.45 == 0)
opt.add(c - f - e - d == 0)
opt.add(d <= 250)
opt.add(e <= 250)

opt.add(cost == If(f > 0, f * 50, f * 0.4) + e * 40 + d * 20 +
  If(g > 0, g * 50, g * 0.54)) # главный критерий -- цена, которую надо минимизировать

opt.minimize(cost) # выполняем оптимизацию по указанному критерию

print(opt.check())
print(opt.model())

Однако с нелинейными проблемами оптимизатор Z3 как правило не справляется, или не находит оптимальный результат.

In [ ]:
l = Real("l")
w = Real("w")
b = Real("b")
opt = Optimize()
width = 63.6
height = 51

opt.add(b+l <= width)
opt.add(w+b+w+l+w <= height)
opt.add(w > 0)
opt.add(b > 0)
opt.add(l > 0)
opt.maximize(w * b * l)

print(opt.check())
print(opt.model())
print(opt.reason_unknown())

# 5. Множественные солверы


При решении сложной задачи можно использовать несколько солверов одновременно. При этом передача условий и формул между ними тривиальна.

In [ ]:
x, y = Ints('x y')

s1 = Solver() # первый солвер
s1.add(x > 10, y > 10)
print (s1)

s2 = Solver() # второй солвер
print (s2) # пусто

s2.add(s1.assertions()) # копируем условия из первого солвера во второй
print (s2)